In [1]:
import pandas as pd

# read in the HCP data
hcp_data = pd.read_csv('/kaggle/input/hcp-data/HCP_Data_KDAG_Hackathon.csv')

# read in the patient data
patient_data = pd.read_csv('/kaggle/input/hcp-data/Patient_Data_KDAG_Hackathon.csv')



# read in the HCP data
hcp_data = pd.read_csv('/kaggle/input/hcp-data/HCP_Data_KDAG_Hackathon.csv')

# convert columns with large data types to smaller data types
hcp_data['Time_Period'] = pd.to_numeric(hcp_data['Time_Period'], downcast='integer')
hcp_data['Brand_Rx'] = pd.to_numeric(hcp_data['Brand_Rx'], downcast='float')
hcp_data['Market_Rx'] = pd.to_numeric(hcp_data['Market_Rx'], downcast='float')
hcp_data['Sales_Rep_Calls'] = pd.to_numeric(hcp_data['Sales_Rep_Calls'], downcast='integer')
hcp_data['Samples_Dropped'] = pd.to_numeric(hcp_data['Samples_Dropped'], downcast='integer')
hcp_data['Emails_Delivered'] = pd.to_numeric(hcp_data['Emails_Delivered'], downcast='integer')
hcp_data['Speaker_Programs_Attended'] = pd.to_numeric(hcp_data['Speaker_Programs_Attended'], downcast='integer')
hcp_data['Vouchers_Dropped'] = pd.to_numeric(hcp_data['Vouchers_Dropped'], downcast='integer')

# read in the patient data
patient_data = pd.read_csv('/kaggle/input/hcp-data/Patient_Data_KDAG_Hackathon.csv')

# convert columns with large data types to smaller data types
patient_data['Patient ID'] = pd.to_numeric(patient_data['Patient ID'], downcast='integer')
patient_data['Year of Birth'] = pd.to_numeric(patient_data['Year of Birth'], downcast='integer')
# patient_data['Number_of_Patients'] = pd.to_numeric(patient_data['Number_of_Patients'], downcast='integer')



In [2]:
hcp_data

,Physician_ID,Time_Period,Brand_Rx,Market_Rx,Sales_Rep_Calls,Samples_Dropped,Physician_Segment,Emails_Delivered,Speaker_Programs_Attended,Vouchers_Dropped,Specialty
0,axt00001,20190104,0.0,2.80567,1,0,3-Low,0,0,0,Dermatologist
1,axt00001,20190111,0.0,20.57312,1,0,3-Low,0,0,0,Dermatologist
2,axt00001,20190118,0.0,6.16010,1,0,3-Low,0,0,0,Dermatologist
3,axt00001,20190125,0.0,8.95501,1,5,3-Low,0,0,0,Dermatologist
4,axt00001,20190201,0.0,9.13793,1,0,3-Low,0,0,0,Dermatologist
...,...,...,...,...,...,...,...,...,...,...,...
569995,axt10000,20200103,0.0,2.45597,0,0,3-Low,1,0,0,Dermatologist
569996,axt10000,20200110,0.0,2.38065,0,0,3-Low,0,0,0,Dermatologist
569997,axt10000,20200117,0.0,7.42171,0,0,3-Low,1,0,0,Dermatologist
569998,axt10000,20200124,0.0,5.39345,0,0,3-Low,1,0,0,Dermatologist


In [3]:
# merge the two data sets on the physician ID
merged_data = pd.merge(hcp_data, patient_data, on='Physician_ID')


In [4]:
# convert the 'Time Period' column to a datetime data type
merged_data['Time_Period'] = pd.to_datetime(merged_data['Time_Period'])


In [5]:
# handle missing values and encode categorical variables
# merged_data = merged_data.fillna(0)
merged_data = pd.get_dummies(merged_data, columns=['Specialty', 'Gender', 'Geographical_State'])


In [6]:
# split the data into training and test sets
train = merged_data.loc[merged_data['Time_Period'] < '2020-01-31']
test = merged_data.loc[merged_data['Time_Period'] == '2020-02-07']


In [13]:
# separate the features and target variable
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# select numerical and categorical columns
num_cols = ['Market_Rx', 'Sales_Rep_Calls', 'Samples_Dropped', 'Emails_Delivered', 'Speaker_Programs_Attended', 'Vouchers_Dropped']
cat_cols = ['Physician_Segment', 'Specialty']

# create the preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)
    ])
X_train = train.drop(['Brand_Rx', 'Time_Period', 'Physician_ID'], axis=1)
y_train = train['Brand_Rx']
X_test = test.drop(['Brand_Rx', 'Time_Period', 'Physician_ID'], axis=1)
y_test = test['Brand_Rx']


In [11]:

# preprocess the training and test data
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)


ValueError: A given column is not a column of the dataframe

In [ ]:
from sklearn.linear_model import LinearRegression

# create the model
model = LinearRegression()

# fit the model to the training data
model.fit(X_train_processed, y_train)


In [ ]:
import numpy as np

# create a dataframe with the predictions and promotional constraints for each HCP
prediction_df = pd.DataFrame({'Predicted_Brand_Rx': predictions, 'Sales_Rep_Calls': test['Sales_Rep_Calls'], 'Samples_Dropped': test['Samples_Dropped'], 'Emails_Delivered': test['Emails_Delivered']})


In [ ]:
# create an empty list to store the chosen channels
chosen_channels = []

# for each HCP
for index, row in prediction_df.iterrows():
    # initialize the best channel to 'none'
    best_channel = 'None'
    # initialize the highest predicted Brand Rx to 0
    highest_prediction = 0
    # if sales rep calls are allowed and the predicted Brand Rx is higher than the current highest prediction
    if row['Sales_Rep_Calls'] == 1 and row['Predicted_Brand_Rx'] > highest_prediction:
        # set the best channel to 'sales rep calls'
        best_channel = 'Sales Rep Calls'
        # set the highest prediction to the current predicted Brand Rx
        highest_prediction = row['Predicted_Brand_Rx']
    # if sample drops are allowed and the predicted Brand Rx is higher than the current highest prediction
    if row['Samples_Dropped'] > 0 and row['Predicted_Brand_Rx'] > highest_prediction:
        # set the best channel to 'sample drops'
        best_channel = 'Sample Drops'
        # set the highest prediction to the current predicted Brand Rx
        highest_prediction = row['Predicted_Brand_Rx']
    # if emails are allowed and the predicted Brand Rx is higher than the current highest prediction
    if row['Emails_Delivered'] > 0 and row['Predicted_Brand_Rx'] > highest_prediction:
        # set the best channel to 'emails'
        best_channel = 'Emails'
        # set the highest prediction to the current predicted Brand Rx
        highest_prediction = row['Predicted_Brand_Rx']
    # add the chosen channel to the list
    chosen_channels.append(best_channel)

# print the chosen channels
print(chosen_channels)
